#### Задача 1. 

С помощью модуля time реализуйте декоратор, который будет высчитывать время выполнения функции.

In [7]:
from time import time

'''decorator'''
def timing(func):
    def wrapper(*args, **kwargs):
        start_time = time()
        res = func(*args, **kwargs)
        end_time = time()
        exec_time = end_time - start_time
        print(f'Время выполнения функции {func.__name__}: {exec_time:.5f} сек.')
        return res
    return wrapper

In [8]:
@timing
def example_func(n):
    for i in range(1, n):
        pass

example_func(10000000)

Время выполнения функции example_func: 0.10699 сек.


#### Задача 2. 

Перепишите класс Token с использованием декоратора dataclass.

In [9]:
import pymorphy3
from dataclasses import dataclass, field

In [18]:
class Lemmatize:
    """descriptor"""
    def __get__(self, token, type=None):
        if hasattr(token, '_lemma') and token._lemma is not None: # если есть атрибут _lemma и он не пустой, возвращаем _lemma
            return token._lemma
        else:
            morph = pymorphy3.MorphAnalyzer() # в противном случае — задаём
            token._lemma = morph.parse(token.text)[0].normal_form
            return token._lemma

    def __set__(self, token, value): # нельзя задать атрибут _lemma ручками
        raise AttributeError("Can\'t set!")

@dataclass
class Token:
    # __slots__ = ('text', 'length', 'is_alpha', 'is_punct', 'is_digit', '_lemma')
    # token_count = 0
    # lemma = Lemmatize()
    ''' С помощью field(init=False) убираем поля из автоматической инициализации,
    поскольку они будут задаваться потом на основе поля text '''    
    text: str
    length: int = field(init=False)
    is_alpha: bool = field(init=False)
    is_punct: bool = field(init=False)
    is_digit: bool = field(init=False)
    ''' Задаем полю _lemma дефолтное значение, убираем из автомат. инициализации
    и строкового представления (т.к. это скрытый атрибут и он не должен быть виден) '''
    _lemma: str = field(default=None, init=False, repr=False)

    lemma = Lemmatize()
    token_count: int = 0

    # def __init__(self, text, length=None, is_alpha=None, is_punct=None, is_digit=None):
    ''' Инициализируем методы для вычисления length, is_alpha и т.д.'''
    def __post_init__(self):
        self.set_length()
        self.set_is_alpha()
        self.set_is_punct()
        self.set_is_digit()
        Token.increment_token_count()
    
    @classmethod
    def increment_token_count(cls):
        cls.token_count += 1

    @classmethod
    def get_token_count(cls):
        return cls.token_count

    def set_length(self):
        self.length = len(self.text)

    def set_is_alpha(self):
        self.is_alpha = self.text.isalpha()
    
    def set_is_punct(self):
        self.is_punct = all(char in '.,?!:"' for char in self.text)
    
    def set_is_digit(self):
        self.is_digit = self.text.isdigit()

    def __repr__(self):
        return f"Token(text={self.text}, length={self.length}, is_alpha={self.is_alpha}, is_punct={self.is_punct}, is_digit={self.is_digit})\n"

In [19]:
token = Token('стали')
token

Token(text=стали, length=5, is_alpha=True, is_punct=False, is_digit=False)

In [20]:
token.lemma

'стать'

#### Задача 3.

Расширьте декоратор таким образом, чтобы а) элементов в кортеже могло быть сколько угодно и они суммировались все б) декоратор был рекурсивным и умел отыскивать самые вложенные кортежи/списки и складывать их элементы, например: 
[[1, 2], [3, 4]], [[3, 4]] = 17

При этом вложенные числа должны быть все на одном уровне (то есть, не может оказаться, чтобы в списке был и список, и число).

In [28]:
def pairwise_sum_dec(func):
    def wrapper(pairs):
        def recursive_sum(item):
            if isinstance(item, (list, tuple)): # если текущий элемент — список/кортеж, итерируемся по его элементам
                if all(isinstance(subitem, (int, float)) for subitem in item): # если внутри числа, то суммируем их
                    return sum(item)
                elif all(isinstance(subitem, (list, tuple)) for subitem in item): # если внутри списки/кортежи, то снова вызываем recursive_sum()
                    return sum(recursive_sum(subitem) for subitem in item)
                else:
                    raise ValueError('Все элементы должны быть либо числами, либо списками/кортежами.')
            elif isinstance(item, (int, float)): # если текущий элемент — число, возвращаем его
                return item
            else:
                raise ValueError('В функцию можно передать объект, содержащий только числа, списки и/или кортежи.')
        
        sums = [recursive_sum(pair) for pair in pairs] # для каждого pair вызываем recursive_sum(), результаты складываем в список sums
        return func(*sums)
    return wrapper

In [29]:
@pairwise_sum_dec
def summ(*args):
    return sum(args)

res = summ([[[1, 2], [3, 4]], [[3, 4]]])
print(res)

17
